In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc
from typing import Tuple, Optional, Dict
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import nibabel as nib
from einops import rearrange
from scipy import ndimage
import wandb
from pathlib import Path

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
    
from research.data.natural_scenes import (
    NaturalScenesDataset,
)

C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
nsd_path = Path('D:\\Datasets\\NSD')
nsd = NaturalScenesDataset(nsd_path)

In [16]:
with h5py.File(nsd_path / 'derivatives' / 'prepared' / 'nsd_preparation_01.hdf5', 'w') as f:
    for subject_id in tqdm(range(1, 9)):
        subject_name = f'subj0{subject_id}'
        group = f.require_group(subject_name)
        voxel_selection_path = 'derivatives/voxel-selection.hdf5'
        voxel_selection_key = 'nc/value'
        
        voxel_selection_file = h5py.File(nsd.dataset_path / voxel_selection_path, 'r')
        key = f'{subject_name}/{voxel_selection_key}'
        nc = voxel_selection_file[key][:]
        threshold = 5.
        nc = nc[nc > threshold]
        
        
        betas, volume_indices = nsd.load_betas(
            subject_name, 
            voxel_selection_path=voxel_selection_path,
            voxel_selection_key=voxel_selection_key,
            threshold=threshold,
            return_tensor_dataset=False,
            return_volume_indices=True,
        )
        
        responses = nsd.subjects[subject_name]['responses']
        stimulus_ids = responses['73KID'].to_numpy() - 1
        
        num_voxels = volume_indices.shape[0]
        volume_mask = nsd.reconstruct_volume(subject_name, torch.ones(num_voxels), volume_indices).numpy().astype(bool)
        group['betas'] = betas
        group['volume_mask'] = volume_mask
        group['stimulus_ids'] = stimulus_ids
        group['noise_ceiling'] = nc
        

  0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
betas.shape, volume_indices.shape

(torch.Size([27750, 27790]), (27790, 3))

In [9]:
np.ones(5)

array([1., 1., 1., 1., 1.])